In [60]:
import pandas as pd
import matplotlib.pyplot as plt
from ipyleaflet import (Map, Marker, basemaps, basemap_to_tiles,
                        FullScreenControl, LayersControl, LayerGroup,
                        WidgetControl, Circle
                       )
from ipywidgets import HTML, Layout, Dropdown, Output, Textarea, VBox, Label
from datamanager import load_wifi_loc_data
import random

def classify_wifi_strength(rssi):
    """
    Categories for WiFi RSSI Measurements
        Excellent: x >= -50 dbm
        Good: -50 dbm > x >= -60 dbm 
        Fair: -60 dbm > x >= -70 dbm
        Weak: -70 dbm x
        
    Parameters:
        rssi:integer
            The RSSI value
        
    Returns:
        The category of the WiFi signal strength
    """
    if rssi >= -50: return 'Excellent'
    elif rssi < -50 and rssi >= -60: return 'Good'
    elif rssi < -60 and rssi >= -70: return 'Fair'
    return 'Weak'

STRENGTH_COLOUR = {
    # Blue for Excellent
    'Excellent' : '#0A75F0',
    # Green for Good
    'Good' : '#0AF01D',
    # Yellow for Fair
    'Fair' : '#FFFB08',
    # Red for Weak
    'Weak' : '#FF0C08'
}

In [52]:
wifi_data = load_wifi_loc_data('whitfeld-1', alternative_folder='old')
wifi_data

,lat,lon,time,bssid,quality,rssi,essid
0,-31.976879,115.817472,20-09-12-15-48-16,A0:CF:5B:3D:5F:90,42/70,-68,Unifi
1,-31.976879,115.817472,20-09-12-15-48-16,A0:CF:5B:3D:5F:9F,36/70,-74,Unifi
2,-31.976879,115.817472,20-09-12-15-48-16,A0:CF:5B:3D:7F:90,36/70,-74,Unifi
3,-31.976879,115.817472,20-09-12-15-48-16,38:ED:18:D0:EF:6F,31/70,-79,Unifi
4,-31.976879,115.817472,20-09-12-15-48-16,40:CE:24:C4:DB:2F,27/70,-83,Unifi
...,...,...,...,...,...,...,...
84,-31.976379,115.817472,20-09-12-15-50-07,A0:CF:5B:3F:87:EF,50/70,-60,Unifi
85,-31.976329,115.817472,20-09-12-15-50-19,A0:CF:5B:3D:5F:90,38/70,-72,Unifi
86,-31.976329,115.817472,20-09-12-15-50-19,A0:CF:5B:3F:69:30,27/70,-83,Unifi
87,-31.976329,115.817472,20-09-12-15-50-19,A0:CF:5B:3F:87:E0,27/70,-83,Unifi


In [53]:
class WiFiMap:
    
    RADIUS_MODIFIER = 0.5
    CIRCLE_OPACITY = 0.5
    CIRCLE_WEIGHT = 1
    
    def __init__(self, wifi_df, length):
        """
        Initialises the WiFiMap class
        
        Parameters:
            wifi_df: pandas.dataframe
                The Dataframe with the WiFi measurements and locations
        """
        self.wifi_df = wifi_df
        self.current_df = wifi_df
        self.length = length
        
        self.bssid_selector = Dropdown(
            options = ['All Access Points'] + self.wifi_data['bssid'].to_list(),
            layout = Layout(width='auto')
        )
        
        # self.marker_layer = LayerGroup(name='markers')
        self.circle_layer = LayerGroup(name='circles')
        # self.line_layer = LayerGroup(name='lines')
        
        self.map = Map(
            basemap = basemaps.Esri.WorldImagery,
            center = self.get_average_location(),
            zoom = 100
        )
        
        # self.map.add_layer(self.marker_layer)
        self.map.add_layer(self.circle_layer)
        # self.map.add_layer(self.line_layer)
        self.map.add_control(FullScreenControl())
        
        
    def update_map(self):
        self.circle_layer.clear_layers()
        best_strengths = self.current_df.groupby(['lat', 'lon'])['rssi'].max()
        
        for loc, rssi in best_strengths.items():
            colour = STRENGTH_COLOUR[classify_wifi_strength(rssi)]
            circle = Circle(
                location = loc,
                color = colour,
                fill_color = colour,
                radius = int(self.length*WiFiMap.RADIUS_MODIFIER),
                opacity = WiFiMap.CIRCLE_OPACITY,
                weight = WiFiMap.CIRCLE_WEIGHT
            )
            self.circle_layer.add_layer(circle)
    
    def display_map(self):
        self.update_map()
        return self.map
        
    def get_average_location(self, bssid_filter=False):
        data_df = self.current_df
        return(data_df['lat'].mean(), data_df['lon'].mean())

In [54]:
wifi_map = WiFiMap(wifi_data, 5)
wifi_map.display_map()

Map(center=[-31.976632932584273, 115.81747199999998], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [70]:
scan_map = Map(
    basemap=basemaps.Esri.WorldImagery,
    center=(-31.97645513, 115.81831377),
    zoom=100
)

recorded_locations = []

marker = Marker(location=gps_data[0])

scan_map.add_layer(marker)
    
scan_map.add_control(FullScreenControl())

scan_map

Map(center=[-31.97645513, 115.81831377], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_t…

In [95]:
marker.location

[-31.980241664322044, 115.81775844097137]

In [94]:
save = '\n'.join([','.join([str(recorded_locations[i][j]) 
                     for j in range(2)]) for i in range(len(recorded_locations))])

In [92]:
LOCATION = 'test'
with open(GPS_FOLDER + LOCATION + '.csv', 'w') as f:
    f.write(save)

In [19]:
lats = pd.read_excel('./data/gps/coords.xlsx', sheet_name='whitfeld-1', usecols=["Latitude", "Longtitude"])

In [22]:
lats['time'] = pd.DataFrame(data=pd.read_csv('./data/wifi/whitfeld-1/complete.csv', names=['time', 'bssid', 'quality', 'rssi', 'essid'])['time'].unique(), columns=['time'])

In [23]:
lats

,Latitude,Longtitude,time
0,-31.976879,115.817472,20-09-12-15-48-16
1,-31.976829,115.817472,20-09-12-15-48-26
2,-31.976779,115.817472,20-09-12-15-48-36
3,-31.976729,115.817472,20-09-12-15-48-44
4,-31.976679,115.817472,20-09-12-15-48-58
5,-31.976629,115.817472,20-09-12-15-49-07
6,-31.976579,115.817472,20-09-12-15-49-18
7,-31.976529,115.817472,20-09-12-15-49-28
8,-31.976479,115.817472,20-09-12-15-49-42
9,-31.976429,115.817472,20-09-12-15-49-55
